In [1]:
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import polars as pl
import os

In [2]:
df = pd.read_parquet("/kaggle/input/version-5/joey_removed.parquet")

In [3]:
columns_to_drop = ['$insert_id', 'amplitude_id', 'app', 'city', 
                   'device_id', 'event_id', 'event_properties', 
                   'language', 'os_name', 'session_id', 'user_id', 
                   'user_properties', 'rowModel', 'type', 
                   '[Amplitude] Session Replay ID']

df.drop(columns_to_drop, axis=1)

,client_event_time,client_upload_time,country,device_family,dma,event_time,event_type,library,processed_time,region,server_received_time,server_upload_time,slug,displayName,referrer_user,hostname,businessUnit,isInternalUser,roles
0,2024-12-13 19:58:49.905000,2024-12-13 19:58:50.972000,United States,Mac OS X,"Charlotte, NC",2024-12-13 19:58:49.905000,:all-accounts:widget:render,amplitude-ts/2.7.2,2024-12-13 19:58:51.906000,North Carolina,2024-12-13 19:58:50.972000,2024-12-13 19:58:50.974000,None,None,https://accounts.youtube.com/,customer_glslg.federato.ai,[],False,['underwriter']
1,2024-06-25 17:28:46.747000,2024-06-25 17:28:50.441000,United States,Mac OS X,"Houston, TX",2024-06-25 17:28:46.747000,application-window-opened,amplitude-ts/2.7.2,2024-06-25 17:28:51.506000,Texas,2024-06-25 17:28:50.441000,2024-06-25 17:28:50.446000,None,None,None,customer_tozrg.federato.ai,None,False,['underwriter']
2,2024-03-14 10:33:00.511000,2024-03-28 15:04:25.455000,United States,Windows,"Nashville, TN",2024-03-14 10:33:00.511000,account:::view,amplitude-ts/1.8.0,2024-03-28 15:04:26.417000,Tennessee,2024-03-28 15:04:25.455000,2024-03-28 15:04:25.458000,None,None,https://login.microsoftonline.com/,customer_cngpd.federato.ai,None,False,['underwriter']
3,2024-08-07 16:12:43.746000,2024-08-07 16:12:44.882000,United States,Windows,"Des Moines-Ames, IA",2024-08-07 16:12:43.746000,account-lines::configurable-table:render,amplitude-ts/2.7.2,2024-08-07 16:12:45.311000,Iowa,2024-08-07 16:12:44.882000,2024-08-07 16:12:44.884000,None,None,EMPTY,customer_zkcuy.federato.ai,None,False,['underwriter']
4,2023-10-24 15:28:56.740000,2024-03-24 19:55:47.790000,United States,Mac,"Greenville-Spartanburg, SC",2023-10-24 15:28:56.740000,account-lines::documents-and-compliance-accord...,amplitude-ts/1.8.0,2024-03-24 19:55:50.114000,South Carolina,2024-03-24 19:55:47.790000,2024-03-24 19:55:47.793000,None,None,None,customer_jjngo.federato.ai,[],True,"['admin', 'underwriter', 'internal', 'Broker']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2639230,2024-06-13 14:39:00.510000,2024-06-13 14:39:01.613000,United States,Mac OS X,"Greenville-Spartanburg, SC",2024-06-13 14:39:00.510000,::widget:render,amplitude-ts/2.7.2,2024-06-13 14:39:02.026000,North Carolina,2024-06-13 14:39:01.613000,2024-06-13 14:39:01.617000,actions-v2,None,None,customer_zkcuy.federato.ai,[],False,['underwriter']
2639231,2024-11-18 21:17:07.783000,2024-11-18 21:17:07.858000,United States,Mac OS X,"New York, NY",2024-11-18 21:17:07.783000,account-lines::configurable-table:render,amplitude-ts/2.7.2,2024-11-18 21:17:09.887000,New York,2024-11-18 21:17:07.858000,2024-11-18 21:17:07.860000,None,None,https://accounts.google.co.uk/,customer_tozrg.federato.ai,[],False,['underwriter']
2639232,2024-08-05 06:48:52.927000,2024-08-05 06:48:53.075000,India,Windows,None,2024-08-05 06:48:52.927000,account-lines::widget:render,amplitude-ts/2.7.2,2024-08-05 06:49:55.959000,Maharashtra,2024-08-05 06:48:53.075000,2024-08-05 06:48:53.077000,None,None,None,customer_cngpd.federato.ai,None,False,['underwriter']
2639233,2024-07-08 09:48:43.568000,2024-07-08 09:48:44.462000,Poland,Windows,None,2024-07-08 09:48:43.568000,account-lines::widget:render,amplitude-ts/2.7.2,2024-07-08 09:48:46.194000,Lower Silesia,2024-07-08 09:48:44.462000,2024-07-08 09:48:44.463000,account-details-header,Account Details Header,https://login.microsoftonline.com/,customer_qfkqj.federato.ai,None,False,['underwriter']


In [4]:
df['uw'] = df['roles'].str.contains("underwriter")
df['admin'] = df['roles'].str.contains("admin")
df['manager'] = df['roles'].str.contains("manager")
df['broker'] = df['roles'].str.contains("Broker")

In [5]:
df.loc[df['slug'].str.isnumeric(), 'slug'] = 'numeric'

df.loc[df['slug'].str.startswith("account"), 'slug'] = 'account'
df.loc[df['slug'].str.startswith("email-inbox"), 'slug'] = 'email-inbox'
df.loc[df['slug'].str.startswith(("excess", "Excess")), 'slug'] = 'excess'
df.loc[df['slug'].str.startswith("form"), 'slug'] = 'form'
df.loc[df['slug'].str.startswith("limits"), 'slug'] = 'limits'
df.loc[df['slug'].str.startswith("locations"), 'slug'] = 'locations'
df.loc[df['slug'].str.startswith("loss"), 'slug'] = 'loss'
df.loc[df['slug'].str.startswith("policy"), 'slug'] = 'policy'
df.loc[df['slug'].str.startswith("submission"), 'slug'] = 'submission'
df.loc[df['slug'].str.startswith("table"), 'slug'] = 'table'

df.loc[df['slug'].str.contains("access-control"), 'slug'] = 'access-control'
df.loc[df['slug'].str.contains("accounts"), 'slug'] = 'accounts'
df.loc[df['slug'].str.contains("action"), 'slug'] = 'action'
df.loc[df['slug'].str.contains("additional"), 'slug'] = 'additional'
df.loc[df['slug'].str.contains("agency"), 'slug'] = 'agency'
df.loc[df['slug'].str.contains("ah"), 'slug'] = 'ah'
df.loc[df['slug'].str.contains("all-bordereaux"), 'slug'] = 'all-bordereaux'
df.loc[df['slug'].str.contains("auto"), 'slug'] = 'auto'
df.loc[df['slug'].str.contains("aviation|fleet"), 'slug'] = 'aviation'
df.loc[df['slug'].str.contains("bond"), 'slug'] = 'bond'
df.loc[df['slug'].str.contains("broker"), 'slug'] = 'broker'
df.loc[df['slug'].str.contains("classification"), 'slug'] = 'classification'
df.loc[df['slug'].str.contains("construction-excess"), 'slug'] = 'construction-excess'
df.loc[df['slug'].str.contains("contacts|contact-role"), 'slug'] = 'contacts'
df.loc[df['slug'].str.contains("contract"), 'slug'] = 'contract'
df.loc[df['slug'].str.contains("coverages"), 'slug'] = 'coverages'
df.loc[df['slug'].str.contains("dashboard"), 'slug'] = 'dashboard'
df.loc[df['slug'].str.contains("compliance"), 'slug'] = 'compliance'
df.loc[df['slug'].str.contains("driver"), 'slug'] = 'driver'
df.loc[df['slug'].str.contains("enrolled-project-schedule"), 'slug'] = 'enrolled-project-schedule'
df.loc[df['slug'].str.contains("equipment"), 'slug'] = 'equipment'
df.loc[df['slug'].str.contains("financial"), 'slug'] = 'financial'
df.loc[df['slug'].str.contains("flood"), 'slug'] = 'flood'
df.loc[df['slug'].str.contains("fmcsa"), 'slug'] = 'fmcsa'
df.loc[df['slug'].str.contains("gl"), 'slug'] = 'gl'
df.loc[df['slug'].str.contains("general|gen-upsert"), 'slug'] = 'general'
df.loc[df['slug'].str.contains("goals"), 'slug'] = 'goals'
df.loc[df['slug'].str.contains("ifta"), 'slug'] = 'ifta'
df.loc[df['slug'].str.contains("json"), 'slug'] = 'json'
df.loc[df['slug'].str.contains("layer"), 'slug'] = 'layer'
df.loc[df['slug'].str.contains("lob-loss-summary"), 'slug'] = 'lob-loss-summary'
df.loc[df['slug'].str.contains("mech-model|mechanized-model"), 'slug'] = 'mech-model'
df.loc[df['slug'].str.contains("misc-professional"), 'slug'] = 'misc-professional'
df.loc[df['slug'].str.contains("mpl"), 'slug'] = 'mpl'
df.loc[df['slug'].str.contains("mvr"), 'slug'] = 'mvr'
df.loc[df['slug'].str.contains("None|null"), 'slug'] = 'None'
df.loc[df['slug'].str.contains("portfolio"), 'slug'] = 'portfolio'
df.loc[df['slug'].str.contains("prior-carrier"), 'slug'] = 'prior-carrier'
df.loc[df['slug'].str.contains("property"), 'slug'] = 'property'
df.loc[df['slug'].str.contains("regional-manager"), 'slug'] = 'regional-manager'
df.loc[df['slug'].str.contains("reinsurer"), 'slug'] = 'reinsurer'
df.loc[df['slug'].str.contains("role-"), 'slug'] = 'role-'
df.loc[df['slug'].str.contains("rule"), 'slug'] = 'rule'
df.loc[df['slug'].str.startswith("slb"), 'slug'] = 'slb'
df.loc[df['slug'].str.startswith("sov|tiv-"), 'slug'] = 'sov'
df.loc[df['slug'].str.startswith("specialty|Specialty"), 'slug'] = 'specialty'
df.loc[df['slug'].str.startswith("subjectivities"), 'slug'] = 'subjectivities'
df.loc[df['slug'].str.contains("templeton"), 'slug'] = 'templeton'
df.loc[df['slug'].str.contains("tensorflight"), 'slug'] = 'tensorflight'
df.loc[df['slug'].str.contains("terrorism"), 'slug'] = 'terrorism'
df.loc[df['slug'].str.contains("triaged-submission"), 'slug'] = 'triaged-submission'
df.loc[df['slug'].str.contains("ul-schedule"), 'slug'] = 'ul-schedule'
df.loc[df['slug'].str.contains("umbrella"), 'slug'] = 'umbrella'
df.loc[df['slug'].str.contains("upcoming"), 'slug'] = 'upcoming'
df.loc[df['slug'].str.contains("user"), 'slug'] = 'user'
df.loc[df['slug'].str.contains("vehicle"), 'slug'] = 'vehicle'
df.loc[df['slug'].str.contains("winnability"), 'slug'] = 'winnability'
df.loc[df['slug'].str.contains("workers-comp"), 'slug'] = 'workers-comp'

df.loc[df['slug'].str.contains("cat-|flood|earth"), 'slug'] = 'cat'
df.loc[df['slug'].str.contains("-list"), 'slug'] = '-list'
df.loc[df['slug'].str.contains("metric"), 'slug'] = 'metric'
df.loc[df['slug'].str.contains("table"), 'slug'] = 'table'
df.loc[df['slug'].str.contains("rater"), 'slug'] = 'rater'
df.loc[df['slug'].str.contains("form"), 'slug'] = 'form'
df.loc[df['slug'].str.contains("limit|deductible"), 'slug'] = 'lim-ded'
df.loc[df['slug'].str.contains("policy|policies"), 'slug'] = 'policy'
df.loc[df['slug'].str.contains("submission"), 'slug'] = 'submission'
df.loc[df['slug'].str.contains("carrier"), 'slug'] = 'carrier'
df.loc[df['slug'].str.contains("allied-health"), 'slug'] = 'ah'
df.loc[df['slug'].str.contains("email"), 'slug'] = 'email'
df.loc[df['slug'].str.contains("excess|umbrella|large-losses"), 'slug'] = 'excess'
df.loc[df['slug'].str.contains("loss"), 'slug'] = 'loss'
df.loc[df['slug'].str.contains("core-columns|layouts|dynamics-document-checklist|all-rows-dialog|dynamic-iframe-widget|dynamics-timeline|widgets|options-editor|add-form|files-forms|rich-text|built-in|access-control"), 'slug'] = 'formatting?'
df.loc[df['slug'].str.contains("underwriter"), 'slug'] = 'underwriter'
df.loc[df['slug'].str.contains("location|map|routes"), 'slug'] = 'location'
df.loc[df['slug'].str.contains("model|z-scoring|combining-actual-and-expected"), 'slug'] = 'model'
df.loc[df['slug'].str.contains("sov|tiv"), 'slug'] = 'sov'
df.loc[df['slug'].str.contains("casualty"), 'slug'] = 'specialty-casualty'
df.loc[df['slug'].str.contains("misc"), 'slug'] = 'misc'
df.loc[df['slug'].str.contains("fiscal-year|balance-sheet|income-statement|ifta"), 'slug'] = 'financial'
df.loc[df['slug'].str.contains("account"), 'slug'] = 'account'
df.loc[df['slug'].str.contains("codes"), 'slug'] = 'codes'
df.loc[df['slug'].str.contains("fmcsa|mvr"), 'slug'] = 'auto'
df.loc[df['slug'].str.contains("liability|casualty|gl|Casualty"), 'slug'] = 'liability'
df.loc[df['slug'].str.contains("planck|tensorflight|insights"), 'slug'] = 'insight'
df.loc[df['slug'].str.contains("building-details"), 'slug'] = 'property'
df.loc[df['slug'].str.contains("all-bordereaux"), 'slug'] = 'reinsurer'
df.loc[df['slug'].str.contains("guidelines|document"), 'slug'] = 'documentation'
df.loc[df['slug'].str.contains("rating|commodity"), 'slug'] = 'rating-commodity'
df.loc[df['slug'].str.contains("aviation|cyber|terrorism|marine"), 'slug'] = 'specialty'
df.loc[df['slug'].str.contains("mileage|miles"), 'slug'] = 'mileage'
df.loc[df['slug'].str.contains("states"), 'slug'] = 'states'
df.loc[df['slug'].str.contains("form|FORM"), 'slug'] = 'form'

In [6]:
counts = df['slug'].value_counts()
small = counts[counts<=50].index.tolist()
df.loc[df['slug'].isin(small), 'slug'] = 'small'

In [7]:
pd.set_option("display.max_rows", None)
df['slug'].value_counts()

slug
None                          1401498
mpl                            203554
policy                         150021
action                         108459
account                        106549
general                         68721
liability                       66443
property                        52030
location                        50323
my-book                         48182
table                           43960
auto                            36732
submission                      35232
one-drive-link                  25177
attachments                     22500
loss                            19853
excess                          17023
bond                            16948
perils                          12882
goals                           12841
email                           10498
rater                           10304
form                             8189
carrier                          7314
vehicle                          7094
ah                               6884
rating-

In [8]:
df['google'] = df['referrer_user'].str.contains("google")
df['microsoft'] = df['referrer_user'].str.contains("microsoft|teams|office")
df['federato'] = df['referrer_user'].str.contains("federato")
df['uw-portal'] = df['referrer_user'].str.contains("portal")

In [9]:
# convert to datetime
df['client_event_time'] = pd.to_datetime(df['client_event_time'])
df['client_upload_time'] = pd.to_datetime(df['client_upload_time'])
df['event_time'] = pd.to_datetime(df['event_time'])
df['processed_time'] = pd.to_datetime(df['processed_time'])
df['server_received_time'] = pd.to_datetime(df['server_received_time'])
df['server_upload_time'] = pd.to_datetime(df['server_upload_time'])

# separate date and hour columns
df['client_event_date'] = pd.to_datetime(df['client_event_time']).dt.date
df['client_upload_date'] = pd.to_datetime(df['client_upload_time']).dt.date
df['event_date'] = pd.to_datetime(df['event_time']).dt.date
df['processed_date'] = pd.to_datetime(df['processed_time']).dt.date
df['server_received_date'] = pd.to_datetime(df['server_received_time']).dt.date
df['server_upload_date'] = pd.to_datetime(df['server_upload_time']).dt.date

df['client_event_hour'] = pd.to_datetime(df['client_event_time']).dt.hour
df['client_upload_hour'] = pd.to_datetime(df['client_upload_time']).dt.hour
df['event_hour'] = pd.to_datetime(df['event_time']).dt.hour
df['processed_hour'] = pd.to_datetime(df['processed_time']).dt.hour
df['server_received_hour'] = pd.to_datetime(df['server_received_time']).dt.hour
df['server_upload_hour'] = pd.to_datetime(df['server_upload_time']).dt.hour

# other variables created
df['client_check'] = df['client_event_date'] == df['client_upload_date']
# df['server_check'] = df['server_received_date'] == df['server_upload_date'] # all same day
df['idk_check'] = df['event_date'] == df['processed_date']

In [10]:
df = pl.from_pandas(df)

In [11]:
user_session = df.group_by(['user_id', 'session_id']).agg(
    pl.min('client_event_time').alias('min_client_event_time'),
    pl.max('client_event_time').alias('max_client_event_time'),
    pl.col("event_type")
).with_columns( 
    (pl.col('max_client_event_time') - pl.col('min_client_event_time')).alias('session_time')
).with_columns( 
    pl.col("session_time").dt.total_seconds().alias('session_seconds')
)